<a href="https://colab.research.google.com/github/pakjira-srisuk/code-repository/blob/main/Lab_1__Pan_sharpening_6606614771_%E0%B8%A0%E0%B8%B1%E0%B8%84%E0%B8%88%E0%B8%B4%E0%B8%A3%E0%B8%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pan-sharpening**

เทคนิค Pan-sharpening เป็นเทคนิคที่ใช้กับภาพถ่ายดาวเทียม ที่มี Panchromatic band เช่น THEOS-1, LANDSAT-8, LANDSAT-9 เพื่อให้มีรายละเอียดและมีประโยชน์มากขึ้น โดย Lab นี้เราจะทดลองกับข้อมูลดาวเทียม Landsat-9

ข้อมูลถ่ายที่เป็น Multi-Spectral มักจะมีรายละเอียดภาพไม่ค่อยดีนัก เมื่อเทียบกับข้อมูล Panchromatics  ดังนั้น การปรับความคมชัดของภาพช่วยได้โดยการรวมภาพเหล่านี้เข้ากับภาพขาวดำพิเศษที่มีรายละเอียดมากขึ้น ภาพขาวดำที่เรียกว่าภาพแพนโครมาติก มีรายละเอียดในระดับที่สูงกว่า แต่ไม่มีสี เราต้องการใช้รายละเอียดจากภาพแพนโครมาติกและสีจากภาพ Landsat-9 เพื่อสร้างภาพสีที่มีรายละเอียดสูงใหม่

ในการดำเนินการใน Lab นี้ นักศึกษา จำเป็นต้องตรวจสอบให้แน่ใจว่าข้อมูลทั้ง Panchromatic และ Multispectral ของ Landsat-9 อยู่ในตำแหน่งที่สมบูรณ์แบบ เพื่อให้เข้ากันได้อย่างลงตัว

มีหลายวิธีในการรวมภาพเหล่านี้ เช่น Brovey Transform, IHS Transform และ PCA วิธีการเหล่านี้ทำให้แน่ใจว่าภาพใหม่จะคงสีจาก Landsat-9 แต่ยังได้รับรายละเอียดเพิ่มเติมจากภาพแพนโครมาติกด้วย


เมื่อเราทำ Pan-sharpening เสร็จแล้ว เราก็จะได้ภาพใหม่ที่เป็นภาพสีที่มีรายละเอียดสูงขึ้น  โดยมีสีทั้งหมดจาก Landsat-9 และความคมชัดพิเศษจากภาพแบบแพนโครมาติก รูปภาพใหม่นี้สามารถนำไปใช้ได้หลายอย่าง เช่น ศึกษาพื้นดิน ค้นหาการเปลี่ยนแปลง หรือเพียงแค่ดูรายละเอียดเพิ่มเติมเกี่ยวกับโลก ดังนั้น การปรับความคมชัดของภาพเป็นวิธีหนึ่งในการทำให้ภาพจากดาวเทียมมีประโยชน์มากขึ้นสำหรับงานสำคัญทุกประเภท

In [1]:
# ทำการ Authenticate และ initialize Earth Engine
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-pakjira') #อย่าลืมเปลี่ยนชื่อโปรเจคของตัวเอง

In [2]:
# กำหนดพื้นที่สนใจ
geometry = ee.Geometry.Point([98.95799098999555, 18.84423947416328])

# เรียกภาพ L9 ตัวอย่างแล้วดึง RGB และ Pan ออกมา
image = (ee.ImageCollection("LANDSAT/LC09/C02/T1_TOA")
         .filterDate('2022-01-01', '2022-03-30')
         .filterBounds(geometry)
         .sort('CLOUD_COVER')
         .first())

In [3]:
# ทำการ Pan-Sharp
rgb = image.select('B4', 'B3', 'B2')
pan = image.select('B8')

# แปลงเป็น HSV, สลับในแถบ PAN และแปลงกลับเป็น RGB
huesat = rgb.rgbToHsv().select('hue', 'saturation')
upres = ee.Image.cat([huesat, pan]).hsvToRgb()

In [4]:
# สร้างแผนที่
Map = geemap.Map(center=[18.84423947416328, 98.95799098999555], zoom=14)

# แสดง เลเยอร์ ก่อนและหลังโดยใช้พารามิเตอร์ vis เดียวกัน
Map.addLayer(rgb, {'max': 0.28}, 'Original')
Map.addLayer(pan, {'max': 0.28}, 'Pan')
Map.addLayer(upres, {'max': 0.28}, 'Pansharpened')
Map


Map(center=[18.84423947416328, 98.95799098999555], controls=(WidgetControl(options=['position', 'transparent_b…

คำถาม
ข้อเพื่อทดสอบความเข้าใจของนักศึกษาเกี่ยวกับเทคนิคการทำ Pan-sharpening ด้วยภาพ Landsat-9 จงตอบคำถามต่อไปนี้

1. อะไรคือเป้าหมายหลักของการปรับความคมชัดของภาพในการสำรวจระยะไกล โดยเฉพาะเมื่อใช้ภาพ Landsat-9 อธิบายว่าเหตุใดจึงมีความสำคัญในการประมวลผลภาพ

2. อธิบายขั้นตอนสำคัญที่เกี่ยวข้องกับการปรับความคมชัดด้วนเทคนิค Pan-sharpening ตั้งแต่การรับข้อมูลไปจนถึงการสร้างภาพที่ปรับความคมชัด เทคนิค Pan-sharpening มี กระบวนการสุ่มตัวอย่างใหม่ช่วยจัดแนวภาพหลายสเปกตรัมและภาพแพนโครมาติกอย่างไร


คำตอบ
1. เป้าหมายหลักของการทำ Pan-sharpening คือ การรวมข้อดีของภาพ Panchromatic และภาพ Multispectral เข้าด้วยกัน โดยภาพ Panchromatic ของ Landsat-9  มีความละเอียดเชิงพื้นที่สูง 15 เมตร แต่ไม่มีข้อมูลสี ส่วนภาพ Multispectral มีข้อมูลหลายช่วงคลื่นที่ใช้วิเคราะห์ลักษณะของพื้นผิวโลก แต่มีความละเอียด 30 เมตร เป็นความละเอียดที่ต่ำกว่า ดังนั้น การทำ Pan-sharpening จะช่วยเพิ่มความคมชัดของภาพ Multispectral ให้มีความละเอียด 15 เมตร ทำให้เห็นรายละเอียดของพื้นที่ เช่น อาคาร ถนน หรือแปลงเกษตร และเห็นข้อมูลสีได้ชัดเจนขึ้น อีกทั้งยังช่วยให้การแปลภาพ การจำแนกการใช้ที่ดิน และการวิเคราะห์ทรัพยากรธรรมชาติมีความแม่นยำมากขึ้น
2. เทคนิค Pan-sharpening เริ่มต้นขั้นตอน จากการรับข้อมูลภาพดาวเทียม Landsat-9 ซึ่งประกอบด้วยแถบ Multispectral ที่ความละเอียด 30 เมตร และแถบ Panchromatic ที่ความละเอียด 15 เมตร หลังจากนั้นจะเข้าสู่ขั้นตอนการเตรียมข้อมูลเบื้องต้น เช่น การตรวจสอบระบบพิกัดให้ตรงกัน การจัดแนวภาพ การปรับแก้ค่ารังสีหรือแปลงเป็นค่าการสะท้อน และการตัดข้อมูลเฉพาะพื้นที่ศึกษา

   เนื่องจากภาพ Multispectral และ Panchromatic มีความละเอียดเชิงพื้นที่แตกต่างกัน จึงจำเป็นต้องทำ การสุ่มตัวอย่างใหม่ (Resampling) โดยการเพิ่มความละเอียดของภาพ Multispectral จาก 30 เมตรให้เป็น 15 เมตร เพื่อให้มีขนาดพิกเซลเท่ากับภาพ Panchromatic วิธีการที่ใช้ในการสุ่มตัวอย่างใหม่อาจเป็น Nearest Neighbor, Bilinear interpolation หรือ Cubic convolution ซึ่งช่วยให้พิกเซลของทั้งสองชุดข้อมูลสามารถซ้อนทับและจัดแนวกันได้อย่างถูกต้อง ลดความคลาดเคลื่อนเชิงตำแหน่ง
   
   เมื่อข้อมูลมีความละเอียดและตำแหน่งตรงกันแล้ว ทำการผสานข้อมูลด้วยวิธี Pan-sharpening เช่น IHS, Brovey Transform, Principal Component Analysis (PCA) หรือ Gram-Schmidt โดยหลักการคือการนำรายละเอียดเชิงพื้นที่จากแถบ Panchromatic เข้าไปเพิ่มในข้อมูล Multispectral เพื่อให้ได้ภาพที่มีความคมชัดสูงขึ้นโดยยังคงคุณสมบัติทางสเปกตรัม และขั้นตอนสุดท้าย คือ การตรวจสอบคุณภาพของภาพที่ได้ ทั้งในด้านความถูกต้องของสี ความคมชัดของรายละเอียด และความผิดเพี้ยนของข้อมูลสเปกตรัม เพื่อให้มั่นใจว่าผลลัพธ์เหมาะสมต่อการนำไปใช้งานวิเคราะห์